# Modelos Customizados com MLflow
## Introdução

Nesta aula, vamos explorar como criar modelos customizados utilizando o MLflow. Vamos construir um modelo que inclua transformações personalizadas nos dados antes da predição. O objetivo é aprender a:

1. **Criar um modelo customizado utilizando o MLflow utilizando o flavor genérico MLflow pyfunc** .
2. **Registrar o modelo MLflow**.
3. **Carregar o modelo registrado** para reutilização.


### O que é o flavor pyfunc?
O **pyfunc** é um flavor padrão no MLflow que permite empacotar qualquer modelo Python. Isso oferece flexibilidade para realizarmos operações customizadas no momento da predição e ao mesmo tempo tirar proveito da facilidade de servir modelos empacotados pelo MLflow.

## Imports necessários

In [0]:
# Importando as bibliotecas necessárias
import pandas as pd
import mlflow
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Preparando o Dataset

In [0]:
# Carregando o dataset Iris
iris = load_iris()
X = pd.DataFrame(iris.data, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
y = iris.target

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Criando um Modelo Customizado

Nesta etapa, criaremos um modelo Python personalizado. Vamos incluir a previsão de probabilidades ao chamar o método `.predict()`. 

A classe PythonModel do mlflow.pyfunc é utilizada como base.

In [0]:
class CustomModel(mlflow.pyfunc.PythonModel):
    """
    Modelo customizado para integração com o MLflow utilizando a classe `PythonModel`.

    Este modelo encapsula um modelo pré-treinado e fornece
    uma interface para realizar previsões, retornando tanto a classe prevista quanto
    as probabilidades associadas a cada classe.
    """

    def __init__(self, model):
        self.model = model
    
    def predict(self, context, model_input):
        """
        Args:
           context: Contexto do MLflow
           model_input: Dados de entrada para realizar previsões
        Returns:
            pd.DataFrame: Um DataFrame com as seguintes colunas:
                - `predicted_class`: A classe prevista.
                - `probability_class_0`: A probabilidade associada à classe 0.
                - `probability_class_1`: A probabilidade associada à classe 1.
                - `probability_class_2`: A probabilidade associada à classe 2.
        """

        # Converter o input em Dataframe
        if not isinstance(model_input, pd.DataFrame):
          model_input = pd.DataFrame(model_input, columns=self.model.feature_names_in_)
        
        # Obter previsões e probabilidades
        predictions = self.model.predict(model_input)
        probabilities = self.model.predict_proba(model_input)
        
        # Criar um DataFrame com os resultados
        results = pd.DataFrame({
            'predicted_class': predictions,
            'probability_class_0': probabilities[:, 0],
            'probability_class_1': probabilities[:, 1],
            'probability_class_2': probabilities[:, 2]
        })
        
        return results

### Treinando o Modelo Base

In [0]:
base_rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
base_rf_model.fit(X_train, y_train)

Uploading artifacts:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

RandomForestClassifier(random_state=42)

### Registro de um Modelo Customizado no MLflow

Vamos registrar um modelo customizado no MLflow usando a classe `CustomModel`.

O primeiro passo é criar uma instância do modelo customizado. Esse modelo encapsula o modelo base que já foi treinado.

In [0]:
# Criar a instância do modelo customizado
custom_model = CustomModel(base_rf_model)

# Registrar utilizando o mlflow
with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        artifact_path="custom_model",
        python_model=custom_model
    )

2025/01/20 11:39:09 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.15.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

2025/01/20 11:39:10 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/01/20 11:39:11 INFO mlflow.tracking._tracking_service.client: 🏃 View run melodic-crane-0 at: adb-94539347838962.2.azuredatabricks.net/ml/experiments/405272931394742/runs/fab6adb775334c0586383707cd86c8fc.
2025/01/20 11:39:11 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: adb-94539347838962.2.azuredatabricks.net/ml/experiments/405272931394742.


### Testando o modelo Customizado

In [0]:
# Obter a run_id
run_id = run.info.run_id

# Carregar o modelo customizado
loaded_model = mlflow.pyfunc.load_model(f"runs:/{run_id}/custom_model")

sample = X_test.iloc[:1]
print("\nPrevisão do Modelo Customizado incluindo as probabilidades:")
prediction_result = loaded_model.predict(sample)
prediction_result


Previsão do Modelo Customizado incluindo as probabilidades:


,predicted_class,probability_class_0,probability_class_1,probability_class_2
0,1,0.0,0.99,0.01
